In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def peek(data, line=None):
    print(data.shape)
    if line is None:
        print(data.head())
    else:
        print(data.head(line))

In [ ]:
data_list = pickle.load(open("../input/notebook51b2761b51/data_list.pkl","rb"))

In [ ]:
target_list = []
def create_target():
    for shop in range(0,len(data_list)):
        print(shop)
        target_shop = []
        for item in range(0,len(data_list[shop])):
            target = data_list[shop][item]['item_cnt_month']
            target_shop.append(target)
        target_list.append(target_shop)
create_target()

In [ ]:
data_list_clean = []
target_list_clean = []

def clean_data(shift_steps):
    for shop in range(0, len(data_list)):
        print(shop)
        data_list_shop = []
        target_list_shop = []
        for item in range(0, len(data_list[shop])):
            data_clean = data_list[shop][item][:-shift_steps].drop('item_cnt_month', axis = 1)
            target_clean = target_list[shop][item][:-shift_steps]
            data_list_shop.append(data_clean)
            target_list_shop.append(target_clean)
        data_list_clean.append(data_list_shop)
        target_list_clean.append(target_list_shop)
        
clean_data(5)

In [ ]:
train_split_steps = 6
X_train = []
X_test = []
Y_train = []
Y_test = []

def train_test_split(train_split_steps):
    for shop in range(0,len(data_list_clean)):
        print(shop)
        X_train_shop = []
        X_test_shop = []
        Y_train_shop = []
        Y_test_shop = []
        for item in range(0,len(data_list_clean[shop])):
            X_train_shop.append(data_list_clean[shop][item][:-train_split_steps])
            X_test_shop.append(data_list_clean[shop][item][-train_split_steps:])
            Y_train_shop.append(target_list_clean[shop][item][:-train_split_steps])
            Y_test_shop.append(target_list_clean[shop][item][-train_split_steps:])
        X_train.append(X_train_shop)
        X_test.append(X_test_shop)
        Y_train.append(Y_train_shop)
        Y_test.append(Y_test_shop)
        
train_test_split(6)

In [ ]:
num_x_signal = 2
num_y_signal = 1

In [ ]:
model = Sequential()
model.add(GRU(units=10,
             return_sequences=True,
             input_shape=(None, num_x_signal)))
model.add(Dense(num_y_signal, activation="sigmoid"))
model.summary()

In [ ]:
warmup_steps = 4
def loss_mse_warmup(y_true, y_pred):
    y_true_slice = y_true[:,warmup_steps:,:]
    y_pred_slice = y_pred[:,warmup_steps:,:]
    loss = tf.losses.mean_squared_error(y_true=y_true_slice,
                                        y_pred=y_pred_slice)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [ ]:
optimizer = RMSprop(learning_rate=1e-4)
model.compile(loss=loss_mse_warmup, optimizer=optimizer)

In [ ]:
es = EarlyStopping(monitor="val_loss",patience=5,verbose=1)

In [ ]:
def create_shop_data(shop_idx, batch_size=64, sequence_length=12, train=True):
    while True:
        x_batch_scaler = MinMaxScaler()
        y_batch_scaler = MinMaxScaler()

        x_batch_scaler.fit(np.array([X_train[shop_idx][i]['item_price'].unique()[0] for i in range(0,len(X_train[shop_idx]))]).reshape(-1,1))
        y_batch_scaler.fit(np.array([j for i in range(0,len(Y_train[shop_idx])) for j in list(Y_train[shop_idx][i])]).reshape(-1,1))

        x_batch_train_scale = np.zeros((batch_size,sequence_length,num_x_signal))
        x_batch_test_scale = np.zeros((batch_size,6,num_x_signal))

        y_batch_train_scale = np.zeros((batch_size,sequence_length,num_y_signal))
        y_batch_test_scale = np.zeros((batch_size,6,num_y_signal))

        items = np.random.randint(len(X_train[shop_idx]), size=batch_size)
        for idx, item in enumerate(items):
            seq_idx = np.random.randint(X_train[shop_idx][item].shape[0] - sequence_length)
            x_batch_train = X_train[shop_idx][item][seq_idx:seq_idx + sequence_length]
            x_batch_train['item_price'] = x_batch_scaler.transform(np.array(x_batch_train['item_price']).reshape(-1,1))
            x_batch_train_scale[idx] = np.array(x_batch_train)

            x_batch_test = X_test[shop_idx][item]
            x_batch_test['item_price'] = x_batch_scaler.transform(np.array(x_batch_test['item_price']).reshape(-1,1))
            x_batch_test_scale[idx] = np.array(x_batch_test)

            y_batch_train = Y_train[shop_idx][item][seq_idx:seq_idx + sequence_length]
            y_batch_train = y_batch_scaler.transform(np.array(y_batch_train).reshape(-1,1))
            y_batch_train_scale[idx] = np.array(y_batch_train)

            y_batch_test = Y_test[shop_idx][item]
            y_batch_test = y_batch_scaler.transform(np.array(y_batch_test).reshape(-1,1))
            y_batch_test_scale[idx] = np.array(y_batch_test)

        if train:
            yield x_batch_train_scale, y_batch_train_scale
        else:
            yield x_batch_test_scale, y_batch_test_scale

In [ ]:
def train_shop(i, shop_mc):
    history = model.fit_generator(generator=create_shop_data(i),
                                 validation_data=create_shop_data(i,train=False),
                                 validation_steps=50,
                                 steps_per_epoch=1,
                                  epochs=1,
                                  callbacks=[es,shop_mc]
                                 )
    return history

def shop_checkpoint(i):
    mc = ModelCheckpoint("/kaggle/working/Shop." + str(i) + ".h5", monitor="val_loss", mode="min",save_best_only=True)
    return mc

In [ ]:
gpu = tf.config.experimental.list_physical_devices('GPU')
print(gpu)

In [ ]:
tf.test.is_gpu_available()

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())

In [ ]:
history = train_shop(31, shop_checkpoint(31))

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train","test"], loc="best")
plt.show()

In [ ]:
test_data = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
peek(test_data, 2)

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
peek(train, 2)

In [ ]:
ctg = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
peek(ctg, 2)

In [ ]:
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
peek(items, 2)

In [ ]:
train2 = pd.merge(train,items[['item_id','item_category_id']], on = 'item_id')
peek(train2)

In [ ]:
last_price = train2.sort_values(['shop_id','item_id','date']).drop_duplicates(['shop_id','item_id'], keep="last")

In [ ]:
test_data2 = pd.merge(test_data, last_price[['shop_id','item_id','item_price','item_category_id']], on = ['shop_id','item_id'])

In [ ]:
test_data2[(test_data2.shop_id == 31) & (test_data2.item_id == 5037)]

In [ ]:
import collections

item_dict = collections.defaultdict()
for shop in train2['shop_id'].unique():
    items = train2[train2['shop_id'] == shop]
    item_dict[shop] = {}
    for idx,item in enumerate(sorted(items.item_id.unique())):
        item_dict[shop][idx] = item

In [ ]:
def create_predict_monthly_data(shop_index,test_data):
    x_batch_scaler = MinMaxScaler()
    x_batch_scaler.fit(np.array([data_list[shop_index][i]['item_price'].unique()[0] for i in range(0,len(data_list[shop_index]))]).reshape(-1,1))
    x_predict_scale = np.zeros((len(data_list[shop_index]), 6, num_x_signal))
    for item_idx in range(0, len(data_list[shop_index])):
        item = item_dict[shop_index][item_idx]
        nov_data = test_data[(test_data['shop_id'] == shop_index) & (test_data['item_id'] == item)][['item_category_id','item_price']]
        month_data = pd.concat([data_list[shop_index][item_idx].drop('item_cnt_month', axis = 1), nov_data])
        month_data['item_price'] = x_batch_scaler.transform(np.array(month_data['item_price']).reshape(-1,1))
        x_predict_scale[item_idx] = np.array(month_data[-6:])
    return x_batch_scaler, x_predict_scale

def monthly_predictions(shop_index, test_data):
    x_scaler, x_predict_scale = create_predict_monthly_data(shop_index, test_data)
    
    y_scaler = MinMaxScaler()
    y_scaler.fit(np.array([j for i in range(0,len(target_list[shop_index])) for j in list(target_list[shop_index][i])]).reshape(-1,1))
    
    x = x_predict_scale
    
    model = tf.keras.models.load_model("../working/Shop.%d.h5" % (shop_index), custom_objects = {'loss_mse_warmup':loss_mse_warmup})
    
    y_pred = model.predict(x)
    
    y_pred_rescaled = y_scaler.inverse_transform(y_pred.reshape(-1,1))
    y_pred_rescaled = y_pred_rescaled.reshape(y_pred.shape)
    return y_pred_rescaled

In [ ]:
# y_pred_rescaled = monthly_predictions(31, test_data2)

In [ ]:
def map_predictions(shop_idx, y_pred,test_data):
    for item_index in range(0,len(y_pred)):
        item = item_dict[shop_idx][item_index]
        if test_data[(test_data['shop_id'] == shop_idx) & (test_data['item_id'] == item)].shape[0] == 0:
            continue
        else:
            test_data.loc[(test_data['shop_id'] == shop_idx) & (test_data['item_id'] == item), 'pred'] = y_pred[item_index][-1][0]
    return test_data

In [ ]:
# shop31_predict = map_predictions(31, y_pred_rescaled, test_data)

In [ ]:
# shop31_predict[shop31_predict['shop_id'] == 31]